In [156]:
import pandas as pd
import os
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [157]:
ruta_de_acceso = "files\Dataset_de_ventas2"

archivos = os.listdir(ruta_de_acceso)

dataframes ={}

date_format = '%m/%d/%y %H:%M'

for archivo in archivos:
  if archivo.endswith('.csv'): #Asegurarse de que se consideren archivos csv
    mes = archivo.split('_')[3].split('.')[0] #Obtener el nombre del mes sin la extension
    ruta_completa = os.path.join(ruta_de_acceso,archivo) #Ruta de acceso completa
    df = pd.read_csv(ruta_completa) #Cargar el csv en un Dataframe
    df_ventas = df.copy() #Hago copia

    #Transformo todos los valores no numericos en Nan en las columnas Cantidad Pedida y Precio Unitario
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce') #El errors= 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

    #Elimino todos los Nan y me quedo solo con los valores numericos
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

    df_ventas = df_ventas.dropna()

    #Transformo todos los valores numericos al tipo de dato correspondiente
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)

    dataframes[mes] = df_ventas

    dataframes[mes]['Fecha de Pedido'] = pd.to_datetime(dataframes[mes]['Fecha de Pedido'], format=date_format)

In [158]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  return datos

In [159]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

dataframes_copia = dataframes
for mes in meses:
  dataframe_mes = dataframes_copia[mes]
  dataframes_copia[mes] = extraer_datos(dataframe_mes)

In [160]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"

totales_ventas = []

for mes in meses:

  total_ventas_mes = dataframes_copia[mes]['Cantidad Pedida'].sum()
  totales_ventas.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = totales_ventas,
        yaxis= 'y',
        name = 'Unidades Vendidas',
        marker = dict(color= totales_ventas, colorscale='RdYlGn', colorbar=dict(title = 'Total de ventas')),
        text = totales_ventas,
        textposition = 'auto',
)
)

fig.update_layout(
    title = 'Unidades Vendidas por mes',
    xaxis = dict(title='Meses'),
    yaxis = dict(title='Unidades Vendidas',side='left')
)

#Muestro el grafico
fig.show()

In [161]:
import plotly.graph_objects as go

totales_ventas = []

for mes in meses:
  total_ventas_mes = (dataframes_copia[mes]['Cantidad Pedida']*dataframes_copia[mes]['Precio Unitario']).sum().round(2)
  totales_ventas.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = totales_ventas,
        yaxis= 'y',
        name = 'Ingreso Total',
        marker = dict(color= totales_ventas, colorscale='RdYlGn', colorbar=dict(title = 'Total de ventas')),
        text = totales_ventas,
        textposition = 'outside',
)
)

fig.update_layout(
    title = 'Ingreso total por mes',
    xaxis = dict(title='Meses'),
    yaxis = dict(title='Ingreso total',side='left')
)

#Muestro el grafico
fig.show()

In [162]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    Ventas_por_hora = dataframes_copia[mes].groupby('Hora')['Cantidad Pedida'].sum()



    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=Horas,
        y=Ventas_por_hora,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

In [163]:
fig = go.Figure()

horas_analizadas = [12,19]

for hora in horas_analizadas:
  total_productos_vendidos = []

  for mes in meses:
    ventas_hora = dataframes_copia[mes][dataframes_copia[mes]['Hora'] == hora]

    total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].sum())

  # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Scatter(
      x=meses,
      y=total_productos_vendidos,
      mode='lines',
      name=f'{hora} horas',
      line = dict(width=2, shape='spline')
  ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
)

fig.show()

In [164]:
for mes in meses:
    columnas = dataframes_copia[mes]['Dirección de Envio'].str.split(', ', expand=True)
    columnas = columnas.rename(columns={0: 'Calle', 1: 'Ciudad', 2: 'PO BOX'})

    columnas[['Estado', 'Cod Postal']] = columnas['PO BOX'].str.split(' ',expand=True)
    dataframes_copia[mes] = pd.concat([dataframes_copia[mes],columnas],axis=True)

In [165]:
# Agrupo por estado y cantidad de productos vendidos
for mes in meses:
  ventas_por_ciudad = dataframes_copia[mes].groupby('Ciudad')['Cantidad Pedida'].sum()

# Reseteo el indice
ventas_por_ciudad = ventas_por_ciudad.reset_index(inplace=False)
ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,2224
1,Austin,1520
2,Boston,3029
3,Dallas,2300
4,Los Angeles,4373
5,New York City,3828
6,Portland,1859
7,San Francisco,6786
8,Seattle,2202


In [166]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Ciudad",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10)
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()

In [167]:

# Agrupo por estado y cantidad de productos vendidos
for mes in meses:
  ventas_por_estado = dataframes_copia[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado = ventas_por_estado.reset_index(inplace=False)

ventas_por_estado

,Estado,Cantidad Pedida
0,CA,11159
1,GA,2224
2,MA,3029
3,ME,352
4,NY,3828
5,OR,1507
6,TX,3820
7,WA,2202


In [168]:

import plotly.express as px


# Hago un plot de Estados Unidos
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show()

In [169]:
ventas_por_estado_mes = {}
estados = []

fig = go.Figure()

for estado in dataframes_copia['Enero']['Estado']:
  if estado not in estados:
    estados.append(estado)

for mes in meses:
  ventas_por_estado_mes[mes] = dataframes_copia[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado_mes[mes] = ventas_por_estado_mes[mes].reset_index(inplace=False)


# Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Bar(
      x=ventas_por_estado_mes[mes]['Estado'],
      y=ventas_por_estado_mes[mes]['Cantidad Pedida'],
      name=f'{mes}',
  ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por mes por estado",
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Ventas", side="left"),
)

fig.show()

In [170]:
# Agrupo por estado y cantidad de productos vendidos
for mes in meses:
  producto_mas_vendido = dataframes_copia[mes].groupby('Producto')['Cantidad Pedida'].sum()
  producto_mas_vendido = producto_mas_vendido.reset_index(inplace=False)

producto_mas_vendido

,Producto,Cantidad Pedida
0,20in Monitor,571
1,27in 4K Gaming Monitor,861
2,27in FHD Monitor,965
3,34in Ultrawide Monitor,849
4,AA Batteries (4-pack),3718
5,AAA Batteries (4-pack),4240
6,Apple Airpods Headphones,2079
7,Bose SoundSport Headphones,1825
8,Flatscreen TV,665
9,Google Phone,716


In [171]:
fig = go.Figure()

productos = producto_mas_vendido['Producto']
cantidad_producto = producto_mas_vendido['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Bar(
    x=productos,
    y=cantidad_producto,
    name="Productos",
    marker = dict(color= cantidad_producto, colorscale='RdYlGn', colorbar=dict(title = 'Total de ventas')),
))

# Configurar ejes y título
fig.update_layout(
    title="Cantidad vendida de productos",
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()

In [172]:
ventas_producto_mes = {}

fig = go.Figure()

for mes in meses:
  ventas_producto_mes[mes] = dataframes_copia[mes].groupby('Producto')['Cantidad Pedida'].sum()
  ventas_producto_mes[mes] = ventas_producto_mes[mes].reset_index(inplace=False)

  # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Bar(
      x=ventas_producto_mes[mes]['Producto'],
      y=ventas_producto_mes[mes]['Cantidad Pedida'],
      name=f'{mes}',
  ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Productos vendidos por mes",
    xaxis=dict(title="Productos"),
    yaxis=dict(title="Ventas", side="left"),
    width=5000,
)

fig.show()

In [182]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

dias = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]



fig = make_subplots(rows=1, cols=12, subplot_titles=meses)

indice = 1;

dataframe_max = 0    
for mes in meses:
    for dia in dias:
        dataframe_dia = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum().max()
        if (dataframe_dia > dataframe_max):
            dataframe_max = dataframe_dia

for mes in meses:
    for dia in dias:
        dataframe_dia = dataframes_copia[mes].groupby('Dia')['Cantidad Pedida'].sum()
        dataframe_dia = dataframe_dia.reset_index(inplace=False)

        # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
        bar_trace1 = go.Bar(
            x=dias,
            y=dataframe_dia['Cantidad Pedida'],
            name=f'{mes}',
        )

        fig.update_xaxes(title_text="Dias", row=1, col=indice)
        fig.update_yaxes(title_text="Cantidad", row=1, col=indice)
        fig.update_yaxes(range=[0, dataframe_max], row=1, col=indice)

    fig.add_trace(bar_trace1, row=1, col=indice)
    indice+=1

# Configuro los ejes y los títulos
fig.update_layout(
    title='Productos por dia por mes',
    width=5000,  # Ancho en píxeles
)

fig.show()

In [219]:
dias_semana=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

fig = make_subplots(rows=1, cols=12, subplot_titles=meses)

indice = 1;

weekday_color = 'blue'
weekend_color = 'red'

max_mes = 0    
for mes in meses:
  for dia in dias:
    dia_de_semana = dataframes_copia[mes].groupby('Fecha de Pedido')['Cantidad Pedida'].sum()
    dia_de_semana = dia_de_semana.reset_index(inplace=False)
    dia_de_semana['Dia de Semana'] = dia_de_semana['Fecha de Pedido'].dt.day_name()

  suma_dia_de_semana = dia_de_semana.groupby('Dia de Semana')['Cantidad Pedida'].sum()
  maximo_mes = suma_dia_de_semana.max()
  if (maximo_mes > max_mes):
    max_mes = maximo_mes

for mes in meses:
  for dia in dias:
    dia_de_semana = dataframes_copia[mes].groupby('Fecha de Pedido')['Cantidad Pedida'].sum()
    dia_de_semana = dia_de_semana.reset_index(inplace=False)
    dia_de_semana['Dia de Semana'] = dia_de_semana['Fecha de Pedido'].dt.day_name()

  suma_dia_de_semana = dia_de_semana.groupby('Dia de Semana')['Cantidad Pedida'].sum()


  # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  bar_trace1 = go.Bar(
      x=dias_semana,
      y=suma_dia_de_semana,
      showlegend=False,
      marker=dict(
        color=[weekday_color if dia in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] else weekend_color for dia in dias_semana])
  )

  fig.update_xaxes(title_text="Dia de semana", row=1, col=indice)
  fig.update_yaxes(title_text="Cantidad", row=1, col=indice)
  fig.update_yaxes(range=[0, max_mes + 100], row=1, col=indice)

  fig.add_trace(bar_trace1, row=1, col=indice)
  indice+=1

# Configuro los ejes y los títulos
fig.update_layout(
    title='Productos por dia de semana por mes',
    width=5000,  # Ancho en píxeles
)

fig.show()

